In [ ]:
# https://www.analyticsvidhya.com/blog/2021/06/amazon-product-review-sentiment-analysis-using-bert/

In [2]:
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
import pandas as pd
import numpy as np

In [ ]:
# num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
# print("Num GPUs Available: ", num_gpus_available)
# assert num_gpus_available > 0 

In [3]:
df_without = pd.read_csv('./without_name/Cleaned_headlines.tsv', sep="\t")
df_without

,뉴스제목,주가변동
0,24 와인 경쟁력 강화로 가맹점 매출 높인다,0
1,ENM 1분기 매출 8108억원 영업이익 397억원,0
2,최신원 회장 '다락휴'로 호텔사업 키운다,0
3,게임박스에 인디게임 3종 출시토종 생태계 키운다,0
4,"신한금투 ""현대차 1분기 영업익 기대치 밑돌 듯""",1
...,...,...
526213,"ET투자뉴스케이씨텍 ""하반기부터 수주 모"" BUY유지-",0
526214,"ET투자뉴스 ""북미회담 쇼크 반영"" MARKETPERFORM-대신증",1
526215,사내 자선경매 수익금 장애 어린이 위해 전액 기부,1
526216,S '5G 월드 어워드' 잇따라 수상 '쾌거',0


In [4]:
headlines = df_without['뉴스제목'].values.tolist()
labels = df_without['주가변동'].tolist()

In [5]:
from sklearn.model_selection import train_test_split

training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(headlines, labels, test_size=.2)

In [6]:
#Assign tokenizer object to the tokenizer class
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [7]:
tokenizer([training_sentences[0]], truncation=True, padding=True, max_length=128)

{'input_ids': [[101, 9989, 50264, 11090, 9485, 13890, 9150, 12965, 90537, 11903, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [8]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)
train_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(train_encodings),
                            training_labels
                            ))
val_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(val_encodings),
                            validation_labels
                            ))

In [9]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
   45/26311 [..............................] - ETA: 37:20:31 - loss: 0.7042 - accuracy: 0.4972

KeyboardInterrupt: 

In [21]:
model.save_pretrained("./saved_models")
loaded_model = TFBertForSequenceClassification.from_pretrained("./saved_models")

Some layers from the model checkpoint at ./saved_models were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./saved_models.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
test_sentence = "This is a really good product. I love it"
predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

In [ ]:
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels])